# Mask R-CNN - Human pose estimation

Inspect and visualize data loading and pre-processing code.

In [1]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

import utils
import visualize
from visualize import display_images
import model as modellib
from model import log

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # 涓嶅叏閮ㄥ崰婊℃樉瀛? 鎸夐渶鍒嗛厤
sess = tf.Session(config=config)
KTF.set_session(sess)  # 璁剧疆session

    

%matplotlib inline 

ROOT_DIR = os.getcwd()
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")



Using TensorFlow backend.


## Configurations

Run one of the code blocks below to import and load the configurations to use.

In [2]:
# MS COCO Dataset
import coco
config = coco.CocoConfig()
COCO_DIR = r"C:\Users\admin\PycharmProjects\Mask_RCNN_Humanpose\data"  # TODO: enter your own path here

## Dataset

In [3]:
# MS COCO Dataset
import coco
config = coco.CocoConfig()
COCO_DIR = r"C:\Users\admin\PycharmProjects\Mask_RCNN_Humanpose\data"  # TODO: enter value here
# Load dataset
assert config.NAME == "coco"
# Training dataset
# load person keypoints dataset
train_dataset_keypoints = coco.CocoDataset(task_type="person_keypoints")
train_dataset_keypoints.load_coco(COCO_DIR, "train")
train_dataset_keypoints.prepare()

#Validation dataset
val_dataset_keypoints = coco.CocoDataset(task_type="person_keypoints")
val_dataset_keypoints.load_coco(COCO_DIR, "val")
val_dataset_keypoints.prepare()

print("Train Keypoints Image Count: {}".format(len(train_dataset_keypoints.image_ids)))
print("Train Keypoints Class Count: {}".format(train_dataset_keypoints.num_classes))
for i, info in enumerate(train_dataset_keypoints.class_info):
    print("{:3}. {:50}".format(i, info['name']))

print("Val Keypoints Image Count: {}".format(len(val_dataset_keypoints.image_ids)))
print("Val Keypoints Class Count: {}".format(val_dataset_keypoints.num_classes))
for i, info in enumerate(val_dataset_keypoints.class_info):
    print("{:3}. {:50}".format(i, info['name']))

loading annotations into memory...
Done (t=11.64s)
creating index...
index created!
Skeleton: (19, 2)
Keypoint names: (17,)
loading annotations into memory...
Done (t=0.38s)
creating index...
index created!
Skeleton: (19, 2)
Keypoint names: (17,)
Train Keypoints Image Count: 64115
Train Keypoints Class Count: 2
  0. BG                                                
  1. person                                            
Val Keypoints Image Count: 2693
Val Keypoints Class Count: 2
  0. BG                                                
  1. person                                            


## Ceate Model

In [4]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)

#'''
init_with = "last"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
    #print("Loading weights from ", model_path)
    print("Loading weights from ", COCO_MODEL_PATH)
    
elif init_with == "last":
    # Load the last model you trained and continue training
    print("Loading weights from ", model.find_last()[1])
    model.load_weights(model.find_last()[1], by_name=True)
    
#'''    
# Load weights trained on MS-COCO
#model.load_weights(COCO_MODEL_PATH, by_name=True,exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
#                                "mrcnn_bbox", "mrcnn_mask"])


# model.keras_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Loading weights from  C:\Users\admin\PycharmProjects\Mask_RCNN_Humanpose\logs\coco20200213T1817\mask_rcnn_coco_0084.h5


In [5]:
# Training - Stage 1
print("Train heads")
'''
model.train(train_dataset_keypoints, val_dataset_keypoints,
            learning_rate=config.LEARNING_RATE,
            epochs=15,
            layers='heads')
'''
'''
# Training - Stage 2
# Finetune layers from ResNet stage 4 and up
print("Training Resnet layer 4+")
model.train(train_dataset_keypoints, val_dataset_keypoints,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=20,
            layers='4+')
'''
#first config.LEARNING_RATE epoch 40 
#2nd  config.LEARNING_RATE / 10  epoch 20

# Training - Stage 3
# Finetune layers from ResNet stage 3 and up
print("Training Resnet layer 3+")
model.train(train_dataset_keypoints, val_dataset_keypoints,     
            #learning_rate=config.LEARNING_RATE / 100,
            learning_rate=config.LEARNING_RATE,
            epochs=100, #100
            layers='all') #all
           

Train heads
Training Resnet layer 3+

Starting at epoch 0. LR=0.002

Checkpoint Path: C:\Users\admin\PycharmProjects\Mask_RCNN_Humanpose\logs\coco20200215T1349\mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
conv1/conv             (Conv2D)
conv1/bn               (BatchNormalization)
conv2_block1_0_bn      (BatchNormalization)
conv2_block1_1_conv    (Conv2D)
conv2_block1_1_bn      (BatchNormalization)
conv2_block1_2_conv    (Conv2D)
conv2_block2_0_bn      (BatchNormalization)
conv2_block2_1_conv    (Conv2D)
conv2_block2_1_bn      (BatchNormalization)
conv2_block2_2_conv    (Conv2D)
conv2_block3_0_bn      (BatchNormalization)
conv2_block3_1_conv    (Conv2D)
conv2_block3_1_bn      (BatchNormalization)
conv2_block3_2_conv    (Conv2D)
conv2_block4_0_bn      (BatchNormalization)
conv2_block4_1_conv    (Conv2D)
conv2_block4_1_bn      (BatchNormalization)
conv2_block4_2_conv    (Conv2D)
conv2_block5_0_bn      (BatchNormalization)
conv2_block5_1_conv    (Conv2D)
conv2_block5_1_bn      (

C:\Users\admin\Anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


Instructions for updating:
Use tf.cast instead.


C:\Users\admin\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
1000/1000 [==============================] - 1589s 2s/step - loss: 4.7294 - rpn_class_loss: 0.0132 - rpn_bbox_loss: 0.2253 - mrcnn_class_loss: 0.1138 - mrcnn_bbox_loss: 0.1714 - keypoint_mrcnn_mask_loss: 3.7975 - mrcnn_mask_loss: 0.4081 - val_loss: 4.1428 - val_rpn_class_loss: 2.9810e-04 - val_rpn_bbox_loss: 0.1938 - val_mrcnn_class_loss: 0.0024 - val_mrcnn_bbox_loss: 0.0336 - val_keypoint_mrcnn_mask_loss: 3.7460 - val_mrcnn_mask_loss: 0.1666
Epoch 2/100
1000/1000 [==============================] - 1504s 2s/step - loss: 4.7692 - rpn_class_loss: 0.0138 - rpn_bbox_loss: 0.2342 - mrcnn_class_loss: 0.1185 - mrcnn_bbox_loss: 0.1739 - keypoint_mrcnn_mask_loss: 3.8240 - mrcnn_mask_loss: 0.4046 - val_loss: 4.3003 - val_rpn_class_loss: 6.9446e-04 - val_rpn_bbox_loss: 0.2351 - val_mrcnn_class_loss: 0.0106 - val_mrcnn_bbox_loss: 0.0336 - val_keypoint_mrcnn_mask_loss: 3.7612 - val_mrcnn_mask_loss: 0.2591
Epoch 3/100
1000/1000 [==============================] - 1511s 2s/step - loss: 4.5

...

Epoch 53/100
1000/1000 [==============================] - 1508s 2s/step - loss: 4.8126 - rpn_class_loss: 0.0140 - rpn_bbox_loss: 0.2361 - mrcnn_class_loss: 0.1111 - mrcnn_bbox_loss: 0.1684 - keypoint_mrcnn_mask_loss: 3.8779 - mrcnn_mask_loss: 0.4048 - val_loss: 3.9724 - val_rpn_class_loss: 2.5188e-04 - val_rpn_bbox_loss: 0.1404 - val_mrcnn_class_loss: 0.0026 - val_mrcnn_bbox_loss: 0.0310 - val_keypoint_mrcnn_mask_loss: 3.6253 - val_mrcnn_mask_loss: 0.1727
Epoch 54/100
1000/1000 [==============================] - 1501s 2s/step - loss: 4.7725 - rpn_class_loss: 0.0136 - rpn_bbox_loss: 0.2386 - mrcnn_class_loss: 0.1130 - mrcnn_bbox_loss: 0.1728 - keypoint_mrcnn_mask_loss: 3.8215 - mrcnn_mask_loss: 0.4130 - val_loss: 4.0317 - val_rpn_class_loss: 8.2913e-04 - val_rpn_bbox_loss: 0.2089 - val_mrcnn_class_loss: 0.0084 - val_mrcnn_bbox_loss: 0.0440 - val_keypoint_mrcnn_mask_loss: 3.5674 - val_mrcnn_mask_loss: 0.2021
Epoch 55/100
1000/1000 [==============================] - 1500s 2s/step - loss: 

Epoch 71/100
1000/1000 [==============================] - 1488s 1s/step - loss: 4.6650 - rpn_class_loss: 0.0124 - rpn_bbox_loss: 0.2178 - mrcnn_class_loss: 0.1113 - mrcnn_bbox_loss: 0.1643 - keypoint_mrcnn_mask_loss: 3.7519 - mrcnn_mask_loss: 0.4072 - val_loss: 3.9046 - val_rpn_class_loss: 7.8792e-05 - val_rpn_bbox_loss: 0.0492 - val_mrcnn_class_loss: 8.3352e-04 - val_mrcnn_bbox_loss: 0.0220 - val_keypoint_mrcnn_mask_loss: 3.6860 - val_mrcnn_mask_loss: 0.1464
Epoch 72/100
1000/1000 [==============================] - 1481s 1s/step - loss: 4.5284 - rpn_class_loss: 0.0108 - rpn_bbox_loss: 0.2347 - mrcnn_class_loss: 0.1018 - mrcnn_bbox_loss: 0.1623 - keypoint_mrcnn_mask_loss: 3.6202 - mrcnn_mask_loss: 0.3984 - val_loss: 3.8247 - val_rpn_class_loss: 1.7572e-04 - val_rpn_bbox_loss: 0.2279 - val_mrcnn_class_loss: 0.0046 - val_mrcnn_bbox_loss: 0.0296 - val_keypoint_mrcnn_mask_loss: 3.4060 - val_mrcnn_mask_loss: 0.1562
Epoch 73/100
1000/1000 [==============================] - 1492s 1s/step - lo

ERROR:root:Error processing image {'id': 202582, 'source': 'coco', 'path': 'C:\\Users\\admin\\PycharmProjects\\Mask_RCNN_Humanpose\\data/train2017\\000000202582.jpg', 'width': 640, 'height': 480, 'annotations': [{'segmentation': [[597.61, 234.84, 598.09, 225.91, 605.33, 225.67, 611.61, 225.91, 615.71, 220.6, 616.68, 212.64, 613.78, 207.33, 610.64, 204.19, 607.99, 205.88, 605.82, 210.95, 609.68, 212.16, 607.75, 216.74, 605.58, 221.33, 595.92, 217.47, 593.27, 231.47, 590.37, 236.05, 589.65, 237.5, 593.99, 236.05, 597.61, 233.64]], 'num_keypoints': 0, 'area': 280.60025, 'iscrowd': 0, 'keypoints': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'image_id': 202582, 'bbox': [589.65, 204.19, 27.03, 33.31], 'category_id': 1, 'id': 199488}, {'segmentation': [[527.95, 217.91, 526.72, 214.63, 528.16, 211.56, 531.02, 209.72, 532.25, 209.51, 533.69, 210.13, 534.1, 211.15, 534.3, 213.2, 534.71,

1000/1000 [==============================] - 1485s 1s/step - loss: 4.6305 - rpn_class_loss: 0.0123 - rpn_bbox_loss: 0.2444 - mrcnn_class_loss: 0.1100 - mrcnn_bbox_loss: 0.1624 - keypoint_mrcnn_mask_loss: 3.7102 - mrcnn_mask_loss: 0.3910 - val_loss: 4.0567 - val_rpn_class_loss: 3.0526e-04 - val_rpn_bbox_loss: 0.1766 - val_mrcnn_class_loss: 0.0013 - val_mrcnn_bbox_loss: 0.0379 - val_keypoint_mrcnn_mask_loss: 3.7139 - val_mrcnn_mask_loss: 0.1266
Epoch 83/100
1000/1000 [==============================] - 1507s 2s/step - loss: 4.7067 - rpn_class_loss: 0.0132 - rpn_bbox_loss: 0.2252 - mrcnn_class_loss: 0.1139 - mrcnn_bbox_loss: 0.1675 - keypoint_mrcnn_mask_loss: 3.7791 - mrcnn_mask_loss: 0.4077 - val_loss: 3.9223 - val_rpn_class_loss: 0.0018 - val_rpn_bbox_loss: 0.1382 - val_mrcnn_class_loss: 0.0018 - val_mrcnn_bbox_loss: 0.0229 - val_keypoint_mrcnn_mask_loss: 3.6157 - val_mrcnn_mask_loss: 0.1417
Epoch 84/100
1000/1000 [==============================] - 1492s 1s/step - loss: 4.7101 - rpn_clas

Epoch 100/100
1000/1000 [==============================] - 1490s 1s/step - loss: 4.6352 - rpn_class_loss: 0.0118 - rpn_bbox_loss: 0.2235 - mrcnn_class_loss: 0.1039 - mrcnn_bbox_loss: 0.1651 - keypoint_mrcnn_mask_loss: 3.7395 - mrcnn_mask_loss: 0.3914 - val_loss: 3.8815 - val_rpn_class_loss: 0.0014 - val_rpn_bbox_loss: 0.1767 - val_mrcnn_class_loss: 0.0019 - val_mrcnn_bbox_loss: 0.0474 - val_keypoint_mrcnn_mask_loss: 3.4571 - val_mrcnn_mask_loss: 0.1969
